In [1]:
# import packages
import pandas as pd
import numpy as np
import sys
import re
import functools

sys.path.append("../tool/")

import preprocess

In [2]:
# load data
df2017 = pd.read_csv("../data/OriginalData/developer_survey_2017/survey_results_public_2017.csv")
df2018 = pd.read_csv("../data/OriginalData/developer_survey_2018/survey_results_public.csv")

/Users/renrui/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# choose the fields
columns = [
    "Professional",
    "University",
    "FormalEducation",
    "Gender",
    "Race",
    "Country",
    "Salary",
    "Currency",
    "CompanySize",
    "DeveloperType",
    "JobSatisfaction",
    "JobSeekingStatus",
    "Methodology",
    "WorkStart",
    "MetricAssess",
    "LastNewJob",
    "SelfTaughtTypes",
    "TimeAfterBootcamp",
    "EducationTypes"
    
]

# field with single value
add_columns = {
    "SalaryType": "Yearly",
    "YearsCoding": np.nan,
    "OpenSource":np.nan,
}

# rename the fields mapper
rename_columns = {
    "EmploymentStatus":"Employment",
    "MajorUndergrad" : "UndergradMajor",
    "WantWorkLanguage": "LanguageDesireNextYear",
    "HaveWorkedLanguage" : "LanguageWorkedWith",
    "WantWorkDatabase" : "DatabaseDesireNextYear",
    "HaveWorkedDatabase" : "DatabaseWorkedWith",
    "HaveWorkedPlatform" : "PlatformWorkedWith",
    "WantWorkPlatform" : "PlatformDesireNextYear",
    "ProgramHobby" : "Hobby",
    "ResumePrompted" : "UpdateCV"
}

In [4]:
# get the dataframe contained the fields choosed
for column in rename_columns.keys():
    columns.append(column)
    
df2017_certain = preprocess.extract_data(df2017, columns)
# df2017.filter(items=columns, axis=1).copy()

# add the new filds
for key, value in add_columns.items():
    df2017_certain[key] = value
    
# rename the columns name
df2017_certain.rename(rename_columns, axis=1, inplace=True)

In [5]:
df2017_certain.head()

,Professional,University,FormalEducation,Gender,Race,Country,Salary,Currency,CompanySize,DeveloperType,...,LanguageWorkedWith,DatabaseDesireNextYear,DatabaseWorkedWith,PlatformWorkedWith,PlatformDesireNextYear,Hobby,UpdateCV,SalaryType,YearsCoding,OpenSource
0,Student,No,Secondary school,Male,White or of European descent,United States,NaN,NaN,NaN,NaN,...,Swift,NaN,NaN,iOS,iOS,"Yes, both",NaN,Yearly,NaN,NaN
1,Student,"Yes, full-time",Some college/university study without earning ...,Male,White or of European descent,United Kingdom,NaN,British pounds sterling (£),20 to 99 employees,NaN,...,JavaScript; Python; Ruby; SQL,MySQL; SQLite,MySQL; SQLite,Amazon Web Services (AWS),Linux Desktop; Raspberry Pi; Amazon Web Servic...,"Yes, both",NaN,Yearly,NaN,NaN
2,Professional developer,No,Bachelor's degree,Male,White or of European descent,United Kingdom,113750.0,British pounds sterling (£),"10,000 or more employees",Other,...,Java; PHP; Python,NaN,MySQL,NaN,NaN,"Yes, both",NaN,Yearly,NaN,NaN
3,Professional non-developer who sometimes write...,No,Doctoral degree,Male,White or of European descent,United States,NaN,NaN,"10,000 or more employees",NaN,...,Matlab; Python; R; SQL,MongoDB; Redis; SQL Server; MySQL; SQLite,MongoDB; Redis; SQL Server; MySQL; SQLite,Windows Desktop; Linux Desktop; Mac OS; Amazon...,Windows Desktop; Linux Desktop; Mac OS; Amazon...,"Yes, both",NaN,Yearly,NaN,NaN
4,Professional developer,No,Master's degree,NaN,NaN,Switzerland,NaN,NaN,10 to 19 employees,Mobile developer; Graphics programming; Deskto...,...,NaN,NaN,NaN,NaN,NaN,"Yes, I program as a hobby",NaN,Yearly,NaN,NaN


In [6]:
# convert the element that is not Male or Female into NoComment
df2017_certain.Gender = df2017_certain.Gender.apply(preprocess.convert_single_func, 
                                                    args=(["Male", "Female"], 
                                                         "NoComment"))
# convert the Race no information value into NoInfo
# regrex pattern
pattern = re.compile(r"I don’t know|I prefer not to say")
df2017_certain.Race = df2017_certain.Race.apply(preprocess.convert_single_func, args=(pattern, "NoInfo", True))

# extract the value in the front of the first ;
df2017_certain.Race = df2017_certain.Race.str.split(";").apply(preprocess.convert_list_funct)


# extract the currecy value that is a alphabet value without a sign
pattern = re.compile(r"(\w*.*\s+\w*)+")
df2017_certain.Currency = df2017_certain.Currency.apply(preprocess.convert_single_func, args=(pattern, False, True))

In [7]:
# convert the COmpanySize no information value into NoInfo
pattern = re.compile(r"(I don't know)|(I prefer not to answer)")
df2017_certain.CompanySize = df2017_certain.CompanySize.apply(preprocess.convert_single_func, args=(pattern, "NoInfo", True))

In [8]:
# convert value about the JobSatisfaction field
def convert_job(x):
    # convert the x value into a string value according by the int value
    
    if x == 0:
        return "Extremely dissatisfied"
    elif x <= 2:
        return "Moderately dissatisfied"
    elif x <= 4:
        return "Slightly dissatisfied"
    elif x == 5:
        return "Neither satisfied nor dissatisfied"
    elif x <= 7:
        return "Slightly satisfied"
    elif x <= 9:
        return "Moderately satisfied"
    elif x == 10:
        return "Extremely satisfied"
    else:
        return x

df2017_certain.JobSatisfaction = df2017_certain.JobSatisfaction.apply(convert_job)

In [9]:
# merge the value about YearsCodedJob and YearsCodedJobPast into the field YearsCoding
years_coding = []
for index, row in df2017[["YearsCodedJob", "YearsCodedJobPast"]].iterrows():
    if not pd.isnull(row.YearsCodedJobPast) and pd.isnull(row.YearsCodedJob):
        years_coding.append(row.YearsCodedJobPast)
    else:
        years_coding.append(row.YearsCodedJob)
        
df2017_certain.YearsCoding = years_coding

In [10]:
# Todo: Fix the values into uniform about YearsCoding field
years_coding_dict = {
    "Less than a year": "0-2 years",
    "1 to 2 years": "0-2 years",
    "2 to 3 years" : "0-2 years",
    "3 to 4 years" : "3-5 years",
    "4 to 5 years" : "3-5 years", 
    "5 to 6 years" : "3-5 years", 
    "6 to 7 years" : "6-8 years", 
    "7 to 8 years" : "6-8 years", 
    "8 to 9 years" : "6-8 years", 
    "9 to 10 years" : "9-11 years",
    "10 to 11 years" : "9-11 years",
    "11 to 12 years" : "9-11 years",
    "12 to 13 years" : "12-14 years",
    "13 to 14 years" : "12-14 years",
    "14 to 15 years" : "12-14 years",
    "15 to 16 years" : "15-17 years",
    "16 to 17 years" : "15-17 years",
    "17 to 18 years" : "15-17 years",
    "18 to 19 years" : "18-20 years",
    "19 to 20 years" : "18-20 years",
}

df2017_certain.YearsCoding.replace(years_coding_dict, inplace=True)

In [11]:
# Todo: fix the value into uniform about DeveloperType field


In [12]:
# Todo: fix the value with redundant space
for pattern, replacement in zip([r"^ | $", r"; ", r" ;"], ["", ";", ";"]):
    for field in ["LanguageDesireNextYear", "LanguageWorkedWith", 
                  "DatabaseWorkedWith", "DatabaseWorkedWith", 
                  "PlatformWorkedWith", "PlatformDesireNextYear",
                 "MetricAssess", "SelfTaughtTypes", "EducationTypes"]:
        
        preprocess.remove_string_space(df2017_certain, pattern, replacement, field, inplace=True)
        
        
#     df2017_certain.LanguageDesireNextYear = \
#         df2017_certain.LanguageDesireNextYear.str.replace(re.compile(pattern), replacement)
#     df2017_certain.LanguageWorkedWith = \
#         df2017_certain.LanguageWorkedWith.str.replace(re.compile(pattern), replacement)
    
#     df2017_certain.DatabaseDesireNextYear = \
#         df2017_certain.DatabaseDesireNextYear.str.replace(re.compile(pattern), replacement)
#     df2017_certain.DatabaseWorkedWith = \
#         df2017_certain.DatabaseWorkedWith.str.replace(re.compile(pattern), replacement)
    
#     df2017_certain.PlatformWorkedWith = \
#         df2017_certain.PlatformWorkedWith.str.replace(re.compile(pattern), replacement)
#     df2017_certain.PlatformDesireNextYear = \
#         df2017_certain.PlatformDesireNextYear.str.replace(re.compile(pattern), replacement)    


In [13]:
# Todo: extract values from the field ProgramHobby to new fields  Hobby and OpenSource
df2017_certain.OpenSource = df2017_certain.Hobby.replace({
    "Yes, I program as a hobby":"No", 
    "Yes, both":"Yes",
    "Yes, I contribute to open source projects":"Yes"
})

df2017_certain.Hobby.replace({
    "Yes, I program as a hobby":"Yes", 
    "Yes, both":"Yes",
    "Yes, I contribute to open source projects":"No"
}, inplace=True)

In [14]:
# Todo: replace value about Not applicable/ never with I've never had a job
df2017_certain.LastNewJob.replace({
    "Not applicable/ never": "I've never had a job"
}, inplace=True)

In [15]:
# Todo: replace value about field TimeAfterBootcamp
df2017_certain.TimeAfterBootcamp.replace({
    "I already had a job as a developer when I started the program" : "I already had a full-time job as a developer when I began the program",
    "Immediately upon graduating":"Immediately after graduating",
    "I haven't gotten a job as a developer yet": "I haven’t gotten a developer job",
}, inplace=True)

In [16]:
df2017_certain.LastNewJob.value_counts(dropna=False).index.difference(df2018.LastNewJob.value_counts(dropna=False).index)

Index([], dtype='object')

In [17]:
df2017_certain.YearsCoding.value_counts(dropna=False)

0-2 years           13302
3-5 years           10576
NaN                  9539
6-8 years            4937
9-11 years           4608
20 or more years     3221
12-14 years          2188
15-17 years          2162
18-20 years           859
Name: YearsCoding, dtype: int64

In [18]:
df2018.YearsCoding.value_counts(dropna=False)

3-5 years           23313
6-8 years           19338
9-11 years          12169
0-2 years           10682
12-14 years          8030
15-17 years          6117
18-20 years          5072
NaN                  5020
30 or more years     3544
21-23 years          2648
24-26 years          1862
27-29 years          1060
Name: YearsCoding, dtype: int64

In [19]:
raise

RuntimeError: No active exception to reraise

In [ ]:
testset = set()
for i in df2017_certain.EducationTypes.str.split(";"):
    if isinstance(i, list):
            for x in i:
                testset.add(x)

In [ ]:
testset

In [ ]:
testset2 = set()
for i in df2018.EducationTypes.str.split(";"):
    if isinstance(i, list):
            for x in i:
                testset2.add(x)

In [ ]:
testset2

In [ ]:
testset2.difference(testset)

In [ ]:
testset.difference(testset2)

In [ ]:
df2017_certain[~(df2017_certain.DeveloperType.str.find(";")>0)].DeveloperType.value_counts()

In [ ]:
df2018[~(df2018.DevType.str.find(";")>0)].DevType.value_counts()

In [ ]:
raise

In [ ]:
df2017_certain.info()

In [20]:
# store the data
df2017_certain.to_csv("../data/survey2017.csv", index=False)